![](../images/rivacon_frontmark_combined_header.png)

# Asian options

In [ ]:
import pandas as pd
import math
import numpy
import random
from scipy.stats import norm
import pyvacon.analytics as analytics
import datetime as dt
import pyvacon.tools.converter as converter
import pyvacon.tools.enums as enums
import pyvacon.marketdata.testdata as mkt_testdata
import pyvacon.marketdata.converter as mkt_converter
import pyvacon.instruments.testdata as ins_testdata
from plotly.offline import init_notebook_mode, iplot
from plotly.graph_objs import *
init_notebook_mode(connected=True) 
import pandas as pd

## Definition of an Asian option

An Asian option is an option where the payoff depends on the arithmetic mean of the underling asset's price during the life of the option. Asian options can be of American or European style. Thus, Asian options are appropriate for corporations that, for example, want to ensure that an average exchange rate over a certain period can be realized.

The payoff of an Asian call option is given as $$max(S_{avg}-K, 0),$$ the payoff of an Asian put option is $$max(K-S_{avg}, 0).$$

The average can thereby be an arithmetic or geometric average.

## Pricing an Asian option – one observation

The price of an (European) Asian option can be calculated applying the Back-76 model which is explained [here](european_plain_vanilla_option.ipynb) with 

$$F_0 = M_1$$ 
and 
$$\sigma^2=\frac{1}{T}\ln\left(\frac{M_2}{M_1^2}\right).$$

This leads to the following modified Black-76 model:

$$c=e^{-rT}[M_1N(d_1)-KN(d_2)] \quad \text{resp.} \quad p=e^{-rT}[KN(-d_2)-M_1N(-d_1)]$$

with $$d_1=\frac{\ln(M_1/K)+(\frac{\frac{1}{T}\ln\left(\frac{M_2}{M_1^2}\right)}{2})T}{\sqrt{\frac{1}{T}\ln\left(\frac{M_2}{M_1^2}\right)}\sqrt{T}} \quad \text{and} \quad d_2 = \frac{\ln(M_1/K)+(\frac{\frac{1}{T}\ln\left(\frac{M_2}{M_1^2}\right)}{2})T}{\sqrt{\frac{1}{T}\ln\left(\frac{M_2}{M_1^2}\right)}\sqrt{T}}= d_1-\sqrt{\frac{1}{T}\ln\left(\frac{M_2}{M_1^2}\right)}\sqrt{T}.$$

When the average is calculated continuously, and $\sigma$, $r$ and $q$ are constant, the first two moments $M_1$ and $M_2$ of the underlying asset's average stock price $S_{avg}$ are

$$M_1 = \frac{e^{(r-q)T}-1}{(r-q)T}S_0,$$

and

$$M_2 = \frac{2e^{[2(r-q)+\sigma^2]T}S_0^2}{(r-q+\sigma^2)(2r-2q+\sigma^2)T^2}+\frac{2S_0^2}{(r-q)T^2}\left(\frac{1}{2(r-q)+\sigma^2}-\frac{e^{(r-q)T}}{r-q+\sigma^2} \right).$$

In [ ]:
def exp(x):
    return math.exp(x)
def sqrt(x):
    return math.sqrt(x)
def cdf(x):
    return norm.cdf(x)
def ln(x):
    return math.log(x)

def AsianOptionPricer_single(_type, S0, K, r_f, r_b_f,r_br_dsc, sigma, T):
    M1 = (exp((r_f-r_b)*T)-1)/((r_f-r_b)*T)*S0
    M2 = (2*exp((2*(r_f-r_b)+sigma**2)*T)*S0**2)/((r_f-r_b+sigma**2)*(2*r_f-2*r_b+sigma**2)*T**2)+(2*S0**2)/((r_f-r_b)*T**2)*((1/(2*(r_f-r_b)+sigma**2))-((exp((r_f-r_b)*T))/(r_f-r_b+sigma**2)))
    sig_n = sqrt(1/T*ln(M2/M1**2))
    d1 = (ln(M1/K)+(sig_n**2/2)*(T))/(sig_n*sqrt(T))
    d2 = d1-sig_n*sqrt(T)
    if _type == 'c':
        cp = 1
    if _type =='p':
        cp = -1
    return cp*exp(-r_dsc*(T))*(M1*cdf(cp*d1)-K*cdf(cp*d2)), M1, M2

In [ ]:
S0 = 50
K = 50
r_dsc = 0.05
r_f = 0.05
r_b = 0.00
sigma = 0.4
T = 1

AsianOptionPricer_single('c', S0, K, r_f, r_b, r_dsc, sigma, T)

## Pricing an Asian option – multiple observations

When the average is calculated from multiple observations at times $T_i$ $(1\leq i\leq m)$, the first two moments of $S_{avg}$, $M_1$ and $M_2$, are 

$$M_1 = \frac{1}{m}\sum_{i=1}^mF_i$$
and
$$M_2 = \frac{1}{m^2}\left(\sum_{i=1}^mF_i^2e^{\sigma_i^2T_i}+2\sum_{j=1}^m\sum_{i=1}^{j-1}F_iF_je^{\sigma_i^2T_i}\right).$$

$F_i$ is the forward price for maturity $T_i$, $\sigma_i$ is the implied volatility for maturity $T_i$.

In [ ]:
def AsianOptionPricer(_type, S0, K, r_f, r_b, r_dsc, sigma_i, T_i, T):
    F = []
    for i in range(len(T_i)):
        F.append(S0*exp((r_f-r_b)*T_i[i]))
    m = len(T_i)
    M1 = sum(F)/m
    sum1 = 0
    for i in range(m):
        sum1 += F[i]**2 * exp(sigma_i[i]**2 * T_i[i])
    sum2 = 0
    for j in range(m):
        for i in range(j):
            sum2 += F[i]*F[j] * exp( sigma_i[i]**2 * T_i[i] )         
    M2 = ( sum1 + 2*sum2 ) / m**2
    sig_n = sqrt(1/T*ln(M2/M1**2))
    d1 = (ln(M1/K)+(sig_n**2/2)*(T))/(sig_n*sqrt(T))
    d2 = d1-sig_n*sqrt(T)
    if _type=='c':
        cp = 1
    if _type =='p':
        cp = -1
    return cp*math.exp(-r_dsc*(T))*(M1*cdf(cp*d1)-K*cdf(cp*d2))    

In [ ]:
S0 = 50
K = 50
r_dsc = 0.05
r_f = 0.05
r_b = 0.00
T = 1
sigma_i = []
T_i = []
x = 0.01
while x <= 1.0:
    T_i.append(x)
    sigma_i.append(0.4)
    x += 0.01

In [ ]:
AsianOptionPricer('c', S0, K, r_f, r_b, r_dsc, sigma_i, T_i, T)

## Plotting the option price

In [ ]:
spots = []

n=0.5
while n<=100:
    spots.append(n)
    n=n+0.1
    
call_prices = []
for i in range(len(spots)):
    call_prices.append(AsianOptionPricer('c', spots[i], K, r_f, r_b, r_dsc, sigma_i, T_i, T))
    
put_prices = []
for i in range(len(spots)):
    put_prices.append(AsianOptionPricer('p', spots[i], K, r_f, r_b, r_dsc, sigma_i, T_i, T))

#Plot the prices
prices_list = {'Spots': spots, 
              'Call Prices': call_prices,
              'Put Prices': put_prices}

prices = pd.DataFrame(prices_list, index = spots)    

prices_plot_data = [Scatter(name = 'Call prices', x = spots, y = prices_list['Call Prices'], mode = 'lines'),
                Scatter(name = 'Put prices', x = spots, y = prices_list['Put Prices'], mode = 'lines'), 
                Scatter(name = 'Strike', x = [K,K], y = [0,max(call_prices)], mode = 'lines')]

layout = Layout(title = "Asian Option Prices",
               xaxis=dict(title ='Spot'),
               yaxis=dict(title = 'Prices'))

iplot(Figure(data=prices_plot_data, layout=layout))

## Asian option pricing using Monte-Carlo simulation

In [ ]:
def MCAsianOptionPricer(Type, S0, K, r_f, r_b, r_dsc, sigma, T,m, n):
    if Type=='c':
         cp = 1
    if Type =='p':
        cp = -1
    PV_total = 0
    Dt = T/m
    for j in range(1,n):
        S_total = 0
        S = S0
        for i in range(m):
            S = S*exp(((r_f-r_b)-sigma**2/2)*Dt+sigma*norm.ppf(random.random())*sqrt(Dt))
            S_total += S
        PV_path = max(cp*(S_total/m-K, 0))*exp(-r_dsc*T)
        PV_total +=PV_path
    return PV_total/n

In [ ]:
S0 = 50
K = 50
r_f = 0.05
r_dsc = 0.05
r_b = 0.00
sigma = 0.4
T = 1
m = 365 # no steps - mit n = 1 sollte es preis von europ. plain vanilla option sein
n =1000 # no of simulations

MCAsianOptionPricer('c', S0, K, r_f, r_b, r_dsc, sigma, T,m, n)

Alternatively, the following code can be used.

In [ ]:
def MC2AsianOptionPricer(T, r_f, r_b, r_dsc, K, sigma, S0, m, n):
    sumavg_path = 0
    for i in range(1,n):
        D = T / m
        S = numpy.random.normal(((r_f-r_b)-sigma**2/2)*D, sigma*D**0.5, m) # normal distributed with mu, sigma and m
        PV_avg_path =  math.exp(-r_dsc*T)*max(numpy.mean(numpy.exp(numpy.cumsum(S)))*S0 - K, 0)
        sumavg_path = sumavg_path+PV_avg_path
    return sumavg_path/n, S

In [ ]:
S0 = 50
K = 50
r_f = 0.05
r_dsc = 0.05
r_b = 0.00
sigma = 0.4
T = 1
m=10000
n = 1000

MC2AsianOptionPricer(T, r_f, r_b, r_dsc, K, sigma, S0, m, n)

# es gibt auch geometric
# american type with binomial tree